# Packing stability problem

In [534]:
from pandas import DataFrame, read_csv
import os
from os import listdir
from os.path import isfile, join
import math
import numpy as np
import random
import copy

In [535]:
file_list = [f for f in listdir(os.getcwd()) if isfile(f) and ('csv' in f)]
file_list

['0-out.csv',
 '1-out.csv',
 '2-out.csv',
 '3-out.csv',
 '4-out.csv',
 '5-out.csv',
 '6-out.csv',
 '7-out.csv',
 '8-out.csv',
 '9-out.csv']

In [536]:
def loadData(file_name):
    data = read_csv(os.getcwd()+'/'+ file_name, sep=',', index_col=False, skiprows = 1, names=['SKU', 'X1', 'Y1', 'Z1', 'X2', 'Y2', 'Z2', 'Aisle', 'Weight'])
    return data
file_name = '0-out.csv'
data = loadData(file_name)
data.head(0)

,SKU,X1,Y1,Z1,X2,Y2,Z2,Aisle,Weight
0,900001,0,0,0,600,400,200,1,12000
1,900001,0,400,0,600,800,200,1,12000
2,900001,600,0,0,1200,400,200,1,12000
3,900001,600,400,0,1200,800,200,1,12000
4,900001,0,0,200,600,400,400,1,12000
5,900001,0,400,200,600,800,400,1,12000
6,900001,600,0,200,1200,400,400,1,12000
7,900001,600,400,200,1200,800,400,1,12000
8,900001,0,0,400,600,400,600,1,12000
9,900001,0,400,400,600,800,600,1,12000


In [556]:
class box():
    def __init__(self, SKU, x_1, y_1, z_1, x_2, y_2, z_2, weight):
        self.SKU = SKU
        self.size = [x_2 - x_1, y_2 - y_1, z_2 - z_1]
        #self.coords = self.size
        #self.rotation = 3
        self.weight = weight
        self.attPoint = [x_1, y_1, z_1]
        self.farCorner = [x_2, y_2, z_2]
    
    #вернуть SKU
    def getSKU(self):
        return self.SKU

    #вернуть длину (x по умолчанию)
    def getLength(self):
        return self.size[0]

    #вернуть ширину (y по умолчанию)
    def getWidth(self):
        return self.size[1]

    #вернуть высоту (z по умолчанию)
    def getHeight(self):
        return self.size[2]
    
    #вернуть периметр основания коробки
    def getPerimeter(self):
        return self.size[0] * 2 + self.size[1] * 2

    #вернуть свои исходные размеры
    def getSize(self):
        return list(self.size)

    #вернуть вес
    def getWeight(self):
        return self.weight

    def __repr__(self):
        return 'Box(SKU=%s, attPoint=%s, farCorner=%s, weight = %s)' % (self.SKU,self.attPoint, self.farCorner, self.weight)

    def getAttPoint(self):
        return list(self.attPoint)

    def getFarCorner(self):
        return list(self.farCorner)

    def getCenterMassX(self):
        return ((self.farCorner[0]+self.attPoint[0]) / 2.0)
    
    def getCenterMassY(self):
        return ((self.farCorner[1]+self.attPoint[1]) / 2.0)
    
    def getCenterMassZ(self):
        return ((self.farCorner[2]+self.attPoint[2]) / 2.0)
        
    def getUpperPlane(self):
        return ((self.attPoint[0],self.attPoint[1], self.farCorner[2]), (self.farCorner[0],self.farCorner[1], self.farCorner[2]))

    def __eq__(self, other):
        if (self.getSKU() == other.getSKU() and self.getFarCorner() == other.getFarCorner() 
                 and self.attPoint() == other.attPoint()):
            return True
        else:
            return False

    def __ne__(self, other):
        if  not (self.farCorner[0] == other.farCorner[0] and 
                 self.farCorner[1] == other.farCorner[1] and 
                 self.farCorner[2] == other.farCorner[2]and 
                 self.attPoint[0] == other.attPoint[0] and 
                 self.attPoint[1] == other.attPoint[1] and 
                 self.attPoint[2] == other.attPoint[2]):
            return True
        else:
            return False

In [557]:
boxes = []

for i in range(0, len(data) - 1):
    #print data['SKU'][i], data['X1'][i], data['Y1'][i], data['Z1'][i], data['X2'][i], data['Y2'][i], data['Z2'][i], data['Weight'][i]
    boxes.append(box(data['SKU'][i], data['X1'][i], data['Y1'][i], data['Z1'][i], data['X2'][i], data['Y2'][i], data['Z2'][i], data['Weight'][i]))
boxes

[Box(SKU=900001, attPoint=[0, 0, 0], farCorner=[600, 400, 200], weight = 12000),
 Box(SKU=900001, attPoint=[0, 400, 0], farCorner=[600, 800, 200], weight = 12000),
 Box(SKU=900001, attPoint=[600, 0, 0], farCorner=[1200, 400, 200], weight = 12000),
 Box(SKU=900001, attPoint=[600, 400, 0], farCorner=[1200, 800, 200], weight = 12000),
 Box(SKU=900001, attPoint=[0, 0, 200], farCorner=[600, 400, 400], weight = 12000),
 Box(SKU=900001, attPoint=[0, 400, 200], farCorner=[600, 800, 400], weight = 12000),
 Box(SKU=900001, attPoint=[600, 0, 200], farCorner=[1200, 400, 400], weight = 12000),
 Box(SKU=900001, attPoint=[600, 400, 200], farCorner=[1200, 800, 400], weight = 12000),
 Box(SKU=900001, attPoint=[0, 0, 400], farCorner=[600, 400, 600], weight = 12000),
 Box(SKU=900001, attPoint=[0, 400, 400], farCorner=[600, 800, 600], weight = 12000),
 Box(SKU=900001, attPoint=[600, 0, 400], farCorner=[1200, 400, 600], weight = 12000),
 Box(SKU=900001, attPoint=[600, 400, 400], farCorner=[1200, 800, 600],

In [558]:
boxess = []
b1 = box(1, 0,0,0,10,10,10, 100)
b2 = box(1, 10,0,0,20,10,10, 100)
boxess.append(b1)
boxess.append(b2)

In [559]:
def getCenterMass(boxes):
    #x_total = boxes[0].getCenterMassX()
    #y_total = boxes[0].getCenterMassY()
    #z_total = boxes[0].getCenterMassZ()
    weight_total = 0
    sum_r_x = 0
    sum_r_y = 0
    sum_r_z = 0
    
    for each in boxes:
        sum_r_x += each.getCenterMassX() * each.getWeight()
        sum_r_y += each.getCenterMassY() * each.getWeight()
        sum_r_z += each.getCenterMassZ() * each.getWeight()
        #x_total = (each.getCenterMassX() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        #y_total = (each.getCenterMassY() * each.getWeight() + y_total * weight_total) / (weight_total + each.getWeight())
        #z_total = (each.getCenterMassZ() * each.getWeight() + x_total * weight_total) / (weight_total + each.getWeight())
        weight_total += each.getWeight()
    
    cm_x = sum_r_x / weight_total
    cm_y = sum_r_y / weight_total
    cm_z = sum_r_z / weight_total
    
    return (cm_x, cm_y, cm_z)

In [560]:
getCenterMass(boxes)

(568.25396825396831, 392.06349206349205, 395.23809523809524)

In [561]:
def perimeter_all(boxess):
    perimeter_all = 0
    for each in boxess:
        perimeter_all += each.getPerimeter()
    return perimeter_all

In [562]:
print perimeter_all(boxes)

34800


In [563]:
def perimeter_common(box1, box2):
    
    def getOverlap(a, b):
            return max(0, min(a[1], b[1]) - max(a[0], b[0]))

    def line_x(box1,box2):
        l_x_1 = (box1.attPoint[0], box1.farCorner[0])
        l_x_2 = (box2.attPoint[0], box2.farCorner[0])
        return getOverlap(l_x_1, l_x_2)

    def line_y(box1,box2):
        l_y_1 = (box1.attPoint[1], box1.farCorner[1])
        l_y_2 = (box2.attPoint[1], box2.farCorner[1])
        return getOverlap(l_y_1, l_y_2)
    
    return ((line_x(box1,box2) + line_y(box1,box2)) * 2) if line_x(box1, box2) * line_y(box1, box2) != 0 else 0

In [564]:
def isSameHeight(box1, box2):
    if box1.attPoint[2] == box2.farCorner[2]:
        return True
    else: 
        return False

In [565]:
def perimeter_com_all(boxes):
    per_com_all = 0
    for each in boxes:
        for each1 in boxes:
            if (each != each1 and isSameHeight(each, each1)):
                per_com_all += perimeter_common(each, each1)
    return per_com_all

In [566]:
print perimeter_com_all(boxes)

28400


In [573]:
def stability_coeff(boxes):
    #if getCenterMassHeight(boxes) < 1000 and (perimeter_all(boxes) - perimeter_com_all(boxes)) > 50:
        #return 1
    #else: 
        #return 0
    return getCenterMass(boxes)[2] + (perimeter_all(boxes) - perimeter_com_all(boxes))

In [574]:
print stability_coeff(boxes)

6795.23809524
